CONEXIÓN CON GOOGLE DRIVE

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


INSTALAMOS LOS PAQUETES NECESARIOS

In [2]:
pip install torch torchvision matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

CARGAMOS LAS LIBRERÍAS NECESARIAS

In [3]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, average_precision_score


FUNCION DE TRANSFORMACIÓN DE IMÁGENES Y CREACIÓN DATASETS Y DATALOADERS

In [4]:
# Transformación para las imágenes
transforma = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(), #pasa la imagen a tensor para que pueda ser procesada
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]) #normalizamos las imágenes
])

# Creamos los datasets
data_dir = '/content/drive/MyDrive/TFM/Lado/dataset' #ruta al dataset
train_data = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=transforma)
val_data = datasets.ImageFolder(os.path.join(data_dir, 'val'), transform=transforma)
test_data = datasets.ImageFolder(os.path.join(data_dir, 'test'), transform=transforma)

# Creamos los dataloaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

ENTRENAMIENTO DEL MODELO BASADO EN RESNET50

In [5]:
#Creamos la variable devide para forzar el uso de gpu en caso de que esté disponible
#ya que de otro modo el entrenamiento llevará mucho tiempo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Cargar modelo preentrenado de Resnet50
model_side = models.resnet50(pretrained=True)

# Reemplazamos la capa final para 2 clases (left, right)

num_ftrs = model_side.fc.in_features # nº activaciones de entrada de última fc layer
model_side.fc = nn.Linear(num_ftrs, 2)  # modificación de última capa fc del modelo
#salida.

# Pasar el modelo a GPU si está disponible
model_side = model_side.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 156MB/s]


In [6]:
# Definimos función de pérdida y optimizador
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_side.parameters(), lr=0.001)

ENTRENAMIENTO DEL MODELO Y GRABACIÓN DE LOS PESOS DEL MEJOR EN BASE A LA PRECISIÓN

In [ ]:
#Creamos la variable de "mejor precisión":

best_acc = 0.0

#Establecemos el número de épocas y comenzamos el bucle de entrenamiento

for epoch in range(10):
    model_side.train() #se pone el modelo en modo entrenamiento
    running_loss = 0.0
    correct = 0 #resultados correctos
    total = 0 #total de resultados

    for inputs, labels in train_loader: #bucle en train_loader
        inputs, labels = inputs.to(device), labels.to(device) #forzamos uso de gpu

        optimizer.zero_grad() #limpiamos los gradientes acumulados
        outputs = model_side(inputs) #pasa las imágenes del train por el modelo
        loss = loss_fn(outputs, labels) #compara resultados con labels
        loss.backward() #backpropagation
        optimizer.step() #aplica los cambios a los pesos del modelo

        running_loss += loss.item()
        max_vals, preds = torch.max(outputs, 1) #Obtenemos las probabilidades
        #y las categorías de lado predichas
        correct += (preds == labels).sum().item() #Acumulamos las predicciones
        #correctas
        total += labels.size(0) #Acumulamos el total de resultados para
        #obtener la precisión.

    epoch_loss = running_loss / len(train_loader) #Pérdida media
    epoch_acc = correct / total #precisión de la época

    print(f'Epoch {epoch + 1}/10 - Loss: {epoch_loss:.4f} - Acc: {epoch_acc:.4f}')

    # Si la precisión mejora, guarda el modelo con mejor precisión.
    if epoch_acc > best_acc:
        best_acc = epoch_acc
        torch.save(model_side.state_dict(), '/content/drive/MyDrive/TFM/modelos/model_side_weights6.pth')
        print(f'Modelo mejorado con precisión: {epoch_acc:.4f}')

print(f'Mejor precisión: {best_acc:.4f}')

CARGA DEL MODELO CON LOS PESOS OPTIMIZADOS

In [7]:
# Cargar modelo con los pesos entrenados
model_side.load_state_dict(torch.load('/content/drive/MyDrive/TFM/modelos/model_side_weights6.pth', map_location=device))

<All keys matched successfully>

EVALUACIÓN DEL MODELO CON EL CONJUNTO DE VALIDACIÓN


In [8]:
model_side.eval() # modelo en modo evaluación
all_preds = []
all_labels = []
all_scores = []

with torch.no_grad():
    for inputs, labels in val_loader: #bucle en val_loader
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model_side(inputs)

        # Obtener las probabilidades
        scores = torch.nn.functional.softmax(outputs, dim=1) #Se obtienen las
        # probabilidades de cada clase
        all_scores.append(scores.cpu().numpy()) #conversión a numpy para
        # almacenar en la lista

        # Obtener la categoría más probable (derecha o izquierda)
        max_vals, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calcular Precisión
accuracy = accuracy_score(all_labels, all_preds)

# Calcular Recall
recall = recall_score(all_labels, all_preds, average='micro')

# Calcular mAP (Mean Average Precision)
all_scores = np.concatenate(all_scores, axis=0)  # Concatenar todas las puntuaciones de los batchs
all_labels = np.array(all_labels) # transformación en one hot encoding
mAP = average_precision_score(np.eye(outputs.size(1))[all_labels], all_scores, average='micro')

print(f'Accuracy: {accuracy * 100:.2f}% - Recall: {recall * 100:.2f}% - mAP: {mAP * 100:.2f}%')

Accuracy: 99.41% - Recall: 99.41% - mAP: 99.94%


EVALUACIÓN DEL MODELO CON EL CONJUNTO DE TEST

In [9]:
model_side.eval() # modelo en modo evaluación
all_preds = []
all_labels = []
all_scores = []

with torch.no_grad():
    for inputs, labels in test_loader: #bucle en test_loader
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model_side(inputs)

        # Obtener las probabilidades
        scores = torch.nn.functional.softmax(outputs, dim=1) #Se obtienen las
        # probabilidades de cada clase
        all_scores.append(scores.cpu().numpy()) #conversión a numpy para
        # almacenar en la lista

        # Obtener la categoría más probable (derecha o izquierda)
        max_vals, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calcular Precisión
accuracy = accuracy_score(all_labels, all_preds)

# Calcular Recall
recall = recall_score(all_labels, all_preds, average='micro')

# Calcular mAP (Mean Average Precision)
all_scores = np.concatenate(all_scores, axis=0)  # Concatenar todas las puntuaciones de los batchs
all_labels = np.array(all_labels) # transformación en one hot encoding
mAP = average_precision_score(np.eye(outputs.size(1))[all_labels], all_scores, average='micro')

print(f'Accuracy: {accuracy * 100:.2f}% - Recall: {recall * 100:.2f}% - mAP: {mAP * 100:.2f}%')

Accuracy: 99.42% - Recall: 99.42% - mAP: 99.97%
